In [ ]:
!pip install pyngrok streamlit tensorflow diffusers torch -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [ ]:
%%writefile model1.py
import streamlit as st
import numpy as np
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from PIL import Image
import tensorflow as tf
import h5py
import json

SEED_SIZE = 100
EMBEDDING_SIZE = 300

def clean_h5_model(file_path):
    with h5py.File(file_path, 'r+') as h5_file:
        model_config = h5_file.attrs['model_config']
        if isinstance(model_config, bytes):
            model_config = model_config.decode('utf-8')
        model_config = json.loads(model_config)

        for layer in model_config['config']['layers']:
            if layer['class_name'] == 'Conv2DTranspose' and 'groups' in layer['config']:
                del layer['config']['groups']
        h5_file.attrs['model_config'] = json.dumps(model_config).encode('utf-8')

@st.cache_resource
def load_gan_model(model_path):
    clean_h5_model(model_path)
    return tf.keras.models.load_model(model_path)

def generate_image(gan_model, text_input):
    noise = tf.random.normal([1, SEED_SIZE])
    words = text_input.lower().split()
    test_embeddings = np.zeros((1, EMBEDDING_SIZE), dtype=np.float32)
    count = 0

    for word in words:
        if word in glove_embeddings:
            test_embeddings[0] += glove_embeddings[word]
            count += 1

    if count > 0:
        test_embeddings[0] /= count

    generated_image = gan_model.predict([noise, test_embeddings], verbose=0)
    generated_image = 0.5 * generated_image + 0.5
    image = Image.fromarray((generated_image[0] * 255).astype(np.uint8))

    # Assign the image to low_quality_image before resizing it
    low_quality_image = image.resize((32, 32), Image.NEAREST)

    return low_quality_image

st.title('Text to Image Generator with GAN')

model_path = '/content/text_to_image_generator_cub_character.h5'
gan_model = load_gan_model(model_path)
st.success("GAN model loaded successfully!")

glove_embeddings = np.load('/content/embedding_data.npy', allow_pickle=True)
glove_embeddings = {item[0]: item[1] for item in glove_embeddings} if isinstance(glove_embeddings, np.ndarray) else glove_embeddings.item()
st.success("GloVe embeddings loaded successfully!")

text_input = st.text_input("What image would you like to create?")

if text_input and gan_model:
    generated_image = generate_image(gan_model, text_input)
    refined_image = modify_image_with_stable_diffusion(generated_image, text_input)
    st.image(refined_image, caption='Generated Image from GAN', use_column_width=True)


Writing model.py


In [ ]:
!ngrok authtoken "2n9XjbtbFcYEkuW3Y0SPoCsfe3J_YHtug7pETtB11smowxz1"

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import threading
from pyngrok import ngrok

port = 8501

public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"")

!streamlit run model.py &

while True:
    pass


 * ngrok tunnel "https://0cc6-34-16-148-255.ngrok-free.app" -> "http://127.0.0.1:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.16.148.255:8501

2024-10-17 15:19:38.029802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-17 15:19:38.069346: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-17 15:19:38.085845: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-17 15:19:39.401673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co